# Complete Fragment Processing Workflow

This notebook performs the complete workflow for processing scATAC-seq fragments across multiple species:
1. **Link fragments** to organized folder structure (by cell type)
2. **Liftover fragments** to human genome (hg38) using parallel implementation
3. **Convert fragments** to coverage bigWig files

---

## Setup and Configuration

Define species, auto-detect cell types, and configure file paths.

In [1]:
# Activate the genomes environment for liftover operations
mamba activate genomes

In [2]:
ls /cluster/work/treutlein/jjans/data/intestine/nhp_atlas/genomes/chain_files

calJac1ToCalJac4.over.chain   hg38ToHg19.over.chain
calJac1ToHg19.over.chain      hg38ToMm10.over.chain
calJac4ToCalJac1.over.chain   hg38ToPanPan2.over.chain
calJac4ToHg38.over.chain      hg38ToPanTro3.over.chain
danRer11ToHg38.over.chain     hg38ToPanTro5.over.chain
danRer7ToDanRer11.over.chain  hg38ToRheMac10.over.chain
gorGor4ToHg38.over.chain      mm10ToHg38.over.chain
hg16ToHg38.over.chain	      obsolete
hg17ToHg38.over.chain	      panPan2ToHg38.over.chain
hg18ToHg38.over.chain	      panTro3ToHg38.over.chain
hg19ToCalJac1.over.chain      panTro5ToHg38.over.chain
hg19ToHg38.over.chain	      panTro6ToHg38.over.chain
hg38ToCalJac4.over.chain      rheMac10ToHg38.over.chain
hg38ToGorGor4.over.chain


In [3]:
# Define species to process
SPECIES=("Human" "Gorilla" "Chimpanzee" "Bonobo" "Macaque" "Marmoset")

# Base directories
BASE_DIR="/cluster/project/treutlein/USERS/jjans/analysis/adult_intestine/atac"
CHAIN_DIR="${BASE_DIR}/genomes/chains"
MARMOSET_CHAIN_DIR="/cluster/work/treutlein/jjans/data/intestine/nhp_atlas/genomes/chain_files"
CHROM_SIZES="/cluster/home/jjanssens/jjans/analysis/cerebellum/genomes_new/homo_sapiens/hg38.chrom.sizes"

# Chain file mapping
declare -A CHAIN_FILES
CHAIN_FILES["Gorilla"]="${CHAIN_DIR}/gorGor4ToHg38.over.chain.gz"
CHAIN_FILES["Chimpanzee"]="${CHAIN_DIR}/panTro5ToHg38.over.chain.gz"
CHAIN_FILES["Bonobo"]="${CHAIN_DIR}/panPan2ToHg38.over.chain.gz"
CHAIN_FILES["Macaque"]="${CHAIN_DIR}/rheMac10ToHg38.over.chain.gz"
CHAIN_FILES["Marmoset_step1"]="${MARMOSET_CHAIN_DIR}/calJac1ToCalJac4.over.chain"
CHAIN_FILES["Marmoset_step2"]="${MARMOSET_CHAIN_DIR}/calJac4ToHg38.over.chain"

echo "✅ Configuration loaded"

✅ Configuration loaded


### Auto-detect Available Cell Types

Scan all species directories to find available cell types.

In [4]:
# Auto-detect cell types from available fragment files across all species
echo "🔍 Scanning for available cell types across all species..."
echo ""

# Create associative array to store unique cell types
declare -A UNIQUE_CELL_TYPES

# Scan each species directory
for species in "${SPECIES[@]}"; do
    species_dir="${BASE_DIR}/consensus_peak_calling_${species}/pseudobulk_bed_files"
    
    if [[ -d "$species_dir" ]]; then
        echo "📂 Scanning ${species}..."
        
        # Find all .fragments.tsv.gz files and extract cell type names
        while IFS= read -r file; do
            if [[ -n "$file" ]]; then
                # Extract cell type name (remove .fragments.tsv.gz extension)
                cell_type=$(basename "$file" .fragments.tsv.gz)
                UNIQUE_CELL_TYPES["$cell_type"]=1
                echo "   ✓ Found: ${cell_type}"
            fi
        done < <(find "$species_dir" -maxdepth 1 -name "*.fragments.tsv.gz" -type f 2>/dev/null)
    else
        echo "⚠️  Directory not found for ${species}: $species_dir"
    fi
    echo ""
done

# Convert associative array keys to regular array
CELL_TYPES=($(echo "${!UNIQUE_CELL_TYPES[@]}" | tr ' ' '\n' | sort))

# Display detected cell types
echo "================================"
echo "📋 Detected Cell Types (${#CELL_TYPES[@]} total):"
echo "================================"
for cell_type in "${CELL_TYPES[@]}"; do
    echo "  • $cell_type"
done
echo ""

# Create a summary of which cell types are present in which species
echo "================================"
echo "📊 Cell Type × Species Matrix:"
echo "================================"
for cell_type in "${CELL_TYPES[@]}"; do
    echo ""
    echo "📌 ${cell_type}:"
    for species in "${SPECIES[@]}"; do
        file_path="${BASE_DIR}/consensus_peak_calling_${species}/pseudobulk_bed_files/${cell_type}.fragments.tsv.gz"
        if [[ -f "$file_path" ]]; then
            echo "   ✅ ${species}"
        else
            echo "   ❌ ${species} (not available)"
        fi
    done
done

echo ""
echo "✅ Cell type detection completed"

🔍 Scanning for available cell types across all species...

📂 Scanning Human...
   ✓ Found: Adipocytes
   ✓ Found: BEST4+_cells
   ✓ Found: Crypt_Fibroblasts_WNT2B+
   ✓ Found: Colonocytes
   ✓ Found: Enterocytes
   ✓ Found: Specialized_Fibroblasts_VCAM1+
   ✓ Found: Villus_Fibroblasts_WNT5B+
   ✓ Found: T_cells
   ✓ Found: Specialized_Fibroblasts_SYNM+
   ✓ Found: Monocytes
   ✓ Found: Plasma_B_cells
   ✓ Found: Stem_cells
   ✓ Found: Tuft_cells
   ✓ Found: TA_cells
   ✓ Found: Specialized_Fibroblasts_RSPO2_3+
   ✓ Found: Specialized_Fibroblasts_RSPO3+_only
   ✓ Found: Pericytes
   ✓ Found: Specialized_Fibroblasts_KCNN3+
   ✓ Found: ILCs
   ✓ Found: Lymphatic_ECs
   ✓ Found: Goblet_cells
   ✓ Found: Macrophages
   ✓ Found: ICCs
   ✓ Found: Myofibroblasts
   ✓ Found: Naive_B_cells
   ✓ Found: EECs
   ✓ Found: Enteric_glia
   ✓ Found: Mast_cells
   ✓ Found: ECs
   ✓ Found: Enteric_neurons
   ✓ Found: MUC6+_cells
   ✓ Found: Paneth_cells

📂 Scanning Gorilla...
   ✓ Found: Specialized_Fibr

---
## Step 1: Link Fragment Files

Create organized directory structure and link fragment files by cell type.

In [4]:
# Create directory structure for each cell type
for cell_type in "${CELL_TYPES[@]}"; do
    cell_type_lower=$(echo "$cell_type" | tr '[:upper:]' '[:lower:]')
    mkdir -p "fragment_files/${cell_type_lower}"
    echo "📁 Created directory: fragment_files/${cell_type_lower}"
done

echo "✅ Directory structure created"

📁 Created directory: fragment_files/adipocytes
📁 Created directory: fragment_files/best4+_cells
📁 Created directory: fragment_files/colonocytes
📁 Created directory: fragment_files/crypt_fibroblasts_wnt2b+
📁 Created directory: fragment_files/ecs
📁 Created directory: fragment_files/eecs
📁 Created directory: fragment_files/enteric_glia
📁 Created directory: fragment_files/enteric_neurons
📁 Created directory: fragment_files/enterocytes
📁 Created directory: fragment_files/eosinophils
📁 Created directory: fragment_files/goblet-like_cells
📁 Created directory: fragment_files/goblet_cells
📁 Created directory: fragment_files/iccs
📁 Created directory: fragment_files/ilcs
📁 Created directory: fragment_files/lymphatic_ecs
📁 Created directory: fragment_files/marco+_lymphatic_ecs
📁 Created directory: fragment_files/muc6+_cells
📁 Created directory: fragment_files/macrophages
📁 Created directory: fragment_files/mast_cells
📁 Created directory: fragment_files/mesothelial_cells
📁 Created directory: fragmen

In [6]:
# Link fragment files for each species and cell type
for cell_type in "${CELL_TYPES[@]}"; do
    cell_type_lower=$(echo "$cell_type" | tr '[:upper:]' '[:lower:]')
    
    for species in "${SPECIES[@]}"; do
        species_lower=$(echo "$species" | tr '[:upper:]' '[:lower:]')
        
        # Source file path
        source_file="${BASE_DIR}/consensus_peak_calling_${species}/pseudobulk_bed_files/${cell_type}.fragments.tsv.gz"
        
        # Target link path
        target_link="fragment_files/${cell_type_lower}/${species_lower}_${cell_type_lower}.fragments.tsv.gz"
        
        # Create symlink if source exists
        if [[ -f "$source_file" ]]; then
            ln -sf "$source_file" "$target_link"
            echo "🔗 Linked: ${species} ${cell_type}"
        else
            echo "⚠️  Source not found: ${species} ${cell_type}"
        fi
    done
done

echo ""
echo "✅ All fragment files linked"

🔗 Linked: Human Adipocytes
🔗 Linked: Gorilla Adipocytes
🔗 Linked: Chimpanzee Adipocytes
🔗 Linked: Bonobo Adipocytes
🔗 Linked: Macaque Adipocytes
🔗 Linked: Marmoset Adipocytes
🔗 Linked: Human BEST4+_cells
🔗 Linked: Gorilla BEST4+_cells
🔗 Linked: Chimpanzee BEST4+_cells
🔗 Linked: Bonobo BEST4+_cells
🔗 Linked: Macaque BEST4+_cells
🔗 Linked: Marmoset BEST4+_cells
🔗 Linked: Human Colonocytes
🔗 Linked: Gorilla Colonocytes
🔗 Linked: Chimpanzee Colonocytes
🔗 Linked: Bonobo Colonocytes
🔗 Linked: Macaque Colonocytes
🔗 Linked: Marmoset Colonocytes
🔗 Linked: Human Crypt_Fibroblasts_WNT2B+
🔗 Linked: Gorilla Crypt_Fibroblasts_WNT2B+
🔗 Linked: Chimpanzee Crypt_Fibroblasts_WNT2B+
🔗 Linked: Bonobo Crypt_Fibroblasts_WNT2B+
🔗 Linked: Macaque Crypt_Fibroblasts_WNT2B+
🔗 Linked: Marmoset Crypt_Fibroblasts_WNT2B+
🔗 Linked: Human ECs
🔗 Linked: Gorilla ECs
🔗 Linked: Chimpanzee ECs
🔗 Linked: Bonobo ECs
🔗 Linked: Macaque ECs
🔗 Linked: Marmoset ECs
🔗 Linked: Human EECs
🔗 Linked: Gorilla EECs
🔗 Linked: Chimpanzee 

In [7]:
# Verify linked files
for cell_type in "${CELL_TYPES[@]}"; do
    cell_type_lower=$(echo "$cell_type" | tr '[:upper:]' '[:lower:]')
    echo "\n📂 Fragment files for ${cell_type}:"
    ls -lh "fragment_files/${cell_type_lower}/"
done

\n📂 Fragment files for Adipocytes:
total 24K
lrwxrwxrwx 1 jjanssens jjanssens-group 147 Nov  4 18:37 bonobo_adipocytes.fragments.tsv.gz -> /cluster/project/treutlein/USERS/jjans/analysis/adult_intestine/atac/consensus_peak_calling_Bonobo/pseudobulk_bed_files/Adipocytes.fragments.tsv.gz
lrwxrwxrwx 1 jjanssens jjanssens-group 151 Nov  4 18:37 chimpanzee_adipocytes.fragments.tsv.gz -> /cluster/project/treutlein/USERS/jjans/analysis/adult_intestine/atac/consensus_peak_calling_Chimpanzee/pseudobulk_bed_files/Adipocytes.fragments.tsv.gz
lrwxrwxrwx 1 jjanssens jjanssens-group 148 Nov  4 18:37 gorilla_adipocytes.fragments.tsv.gz -> /cluster/project/treutlein/USERS/jjans/analysis/adult_intestine/atac/consensus_peak_calling_Gorilla/pseudobulk_bed_files/Adipocytes.fragments.tsv.gz
lrwxrwxrwx 1 jjanssens jjanssens-group 146 Nov  4 18:37 human_adipocytes.fragments.tsv.gz -> /cluster/project/treutlein/USERS/jjans/analysis/adult_intestine/atac/consensus_peak_calling_Human/pseudobulk_bed_files/Adipocy

---
## Step 2: Liftover Fragments to hg38

Use parallel implementation to liftover fragments from each species to the human genome (hg38).

**Important:** This step requires the `liftover_fragments_par.sh` and `liftover_fragments_parchr.sh` scripts.

In [8]:
# Create output directories for lifted fragments (organized by cell type)
for cell_type in "${CELL_TYPES[@]}"; do
    cell_type_lower=$(echo "$cell_type" | tr '[:upper:]' '[:lower:]')
    mkdir -p "lifted_fragments/${cell_type_lower}"
    echo "📁 Created directory: lifted_fragments/${cell_type_lower}"
done

echo "✅ Output directories created"

📁 Created directory: lifted_fragments/adipocytes
📁 Created directory: lifted_fragments/best4+_cells
📁 Created directory: lifted_fragments/colonocytes
📁 Created directory: lifted_fragments/crypt_fibroblasts_wnt2b+
📁 Created directory: lifted_fragments/ecs
📁 Created directory: lifted_fragments/eecs
📁 Created directory: lifted_fragments/enteric_glia
📁 Created directory: lifted_fragments/enteric_neurons
📁 Created directory: lifted_fragments/enterocytes
📁 Created directory: lifted_fragments/eosinophils
📁 Created directory: lifted_fragments/goblet-like_cells
📁 Created directory: lifted_fragments/goblet_cells
📁 Created directory: lifted_fragments/iccs
📁 Created directory: lifted_fragments/ilcs
📁 Created directory: lifted_fragments/lymphatic_ecs
📁 Created directory: lifted_fragments/marco+_lymphatic_ecs
📁 Created directory: lifted_fragments/muc6+_cells
📁 Created directory: lifted_fragments/macrophages
📁 Created directory: lifted_fragments/mast_cells
📁 Created directory: lifted_fragments/mesoth

### Human Fragments (No Liftover Needed)

Human fragments are already in hg38, so we just create a symlink.

In [9]:
# Link Human fragments directly (already in hg38)
for cell_type in "${CELL_TYPES[@]}"; do
    cell_type_lower=$(echo "$cell_type" | tr '[:upper:]' '[:lower:]')
    
    source_file="fragment_files/${cell_type_lower}/human_${cell_type_lower}.fragments.tsv.gz"
    target_link="lifted_fragments/${cell_type_lower}/human_${cell_type_lower}.hg38.fragments.tsv.gz"
    
    if [[ -f "$source_file" ]]; then
        ln -sf "../../${source_file}" "$target_link"
        echo "🔗 Linked Human ${cell_type} (no liftover needed)"
    fi
done

🔗 Linked Human Adipocytes (no liftover needed)
🔗 Linked Human BEST4+_cells (no liftover needed)
🔗 Linked Human Colonocytes (no liftover needed)
🔗 Linked Human Crypt_Fibroblasts_WNT2B+ (no liftover needed)
🔗 Linked Human ECs (no liftover needed)
🔗 Linked Human EECs (no liftover needed)
🔗 Linked Human Enteric_glia (no liftover needed)
🔗 Linked Human Enteric_neurons (no liftover needed)
🔗 Linked Human Enterocytes (no liftover needed)
🔗 Linked Human Goblet_cells (no liftover needed)
🔗 Linked Human ICCs (no liftover needed)
🔗 Linked Human ILCs (no liftover needed)
🔗 Linked Human Lymphatic_ECs (no liftover needed)
🔗 Linked Human MUC6+_cells (no liftover needed)
🔗 Linked Human Macrophages (no liftover needed)
🔗 Linked Human Mast_cells (no liftover needed)
🔗 Linked Human Monocytes (no liftover needed)
🔗 Linked Human Myofibroblasts (no liftover needed)
🔗 Linked Human Naive_B_cells (no liftover needed)
🔗 Linked Human Paneth_cells (no liftover needed)
🔗 Linked Human Pericytes (no liftover needed)

### Standard Species Liftover (Gorilla, Chimpanzee, Bonobo, Macaque)

Single-step liftover using parallel implementation with 30 CPUs.

In [5]:
for cell_type in "${CELL_TYPES[@]}"; do
  ct=$(echo "$cell_type" | tr '[:upper:]' '[:lower:]')
  for species in "${STANDARD_SPECIES[@]}"; do
    sp=$(echo "$species" | tr '[:upper:]' '[:lower:]')
    out="lifted_fragments/${ct}/${sp}_${ct}.hg38.fragments_par.tsv.gz"
    if [[ ! -s "$out" ]] || ! gzip -t "$out" &>/dev/null; then
      echo "TODO: $species $cell_type"
    fi
  done
done

TODO: Chimpanzee Enteric_neurons
TODO: Chimpanzee Eosinophils
TODO: Macaque Eosinophils
TODO: Gorilla Goblet-like_cells
TODO: Chimpanzee Goblet-like_cells
TODO: Bonobo Goblet-like_cells
TODO: Macaque Goblet-like_cells
TODO: Bonobo Goblet_cells
TODO: Macaque Goblet_cells
TODO: Gorilla ICCs
TODO: Chimpanzee ICCs
TODO: Bonobo ICCs
TODO: Macaque ICCs
TODO: Gorilla ILCs
TODO: Chimpanzee ILCs
TODO: Bonobo ILCs
TODO: Macaque ILCs
TODO: Gorilla Lymphatic_ECs
TODO: Chimpanzee Lymphatic_ECs
TODO: Bonobo Lymphatic_ECs
TODO: Macaque Lymphatic_ECs
TODO: Gorilla MARCO+_Lymphatic_ECs
TODO: Chimpanzee MARCO+_Lymphatic_ECs
TODO: Bonobo MARCO+_Lymphatic_ECs
TODO: Macaque MARCO+_Lymphatic_ECs
TODO: Gorilla MUC6+_cells
TODO: Chimpanzee MUC6+_cells
TODO: Bonobo MUC6+_cells
TODO: Macaque MUC6+_cells
TODO: Gorilla Macrophages
TODO: Chimpanzee Macrophages
TODO: Bonobo Macrophages
TODO: Macaque Macrophages
TODO: Gorilla Mast_cells
TODO: Chimpanzee Mast_cells
TODO: Bonobo Mast_cells
TODO: Macaque Mast_cells
TOD

In [5]:
STANDARD_SPECIES=("Gorilla" "Chimpanzee" "Bonobo" "Macaque")  # add "Marmoset" if you want it
# expects: CELL_TYPES array and CHAIN_FILES associative array (declare -A CHAIN_FILES)

is_complete() {
    local f="$1"
    [[ -s "$f" ]] && gzip -t "$f" &>/dev/null
}

for cell_type in "${CELL_TYPES[@]}"; do
    cell_type_lower=$(echo "$cell_type" | tr '[:upper:]' '[:lower:]')

    for species in "${STANDARD_SPECIES[@]}"; do
        species_lower=$(echo "$species" | tr '[:upper:]' '[:lower:]')

        input_file="fragment_files/${cell_type_lower}/${species_lower}_${cell_type_lower}.fragments.tsv.gz"
        chain_file="${CHAIN_FILES[$species]:-}"  # may be empty if not set
        output_file="lifted_fragments/${cell_type_lower}/${species_lower}_${cell_type_lower}.hg38.fragments_par.tsv.gz"
        tmp_file="${output_file}.partial"
        log_file="lifted_fragments/${cell_type_lower}/${species_lower}_${cell_type_lower}.log"

        # Ensure dirs exist
        mkdir -p "$(dirname "$output_file")" "$(dirname "$log_file")"

        # Basic presence checks
        if [[ ! -f "$input_file" ]]; then
            echo "⚠️  Skipping ${species} ${cell_type} (missing input: $input_file)"
            echo ""
            continue
        fi
        if [[ -z "${chain_file}" || ! -f "${chain_file}" ]]; then
            echo "⚠️  Skipping ${species} ${cell_type} (missing chain file mapping for ${species})"
            echo ""
            continue
        fi

        # Already complete? skip
        if is_complete "$output_file"; then
            echo "⏭️  Already done: ${species} ${cell_type} — skipping"
            echo ""
            continue
        fi

        # If an output exists but is invalid, clean it
        if [[ -f "$output_file" ]]; then
            if ! is_complete "$output_file"; then
                echo "🧹 Removing corrupt/partial $output_file"
                rm -f "$output_file"
            fi
        fi
        [[ -f "$tmp_file" ]] && { echo "🧹 Removing leftover $tmp_file"; rm -f "$tmp_file"; }

        echo "🚀 Running liftover for ${species} ${cell_type}..."
        bash ./liftover_fragments_par.sh \
            --i "$input_file" \
            --c "$chain_file" \
            --o "$tmp_file" \
            --ncpu 30 \
            &> "$log_file" || { echo "❌ Liftover failed — see $log_file"; echo ""; continue; }

        if is_complete "$tmp_file"; then
            mv -f "$tmp_file" "$output_file"
            echo "✅ Done with ${species} ${cell_type}"
        else
            echo "❌ Output failed validation for ${species} ${cell_type} — see $log_file"
            rm -f "$tmp_file"
        fi
        echo ""
    done
done

echo "✅ Standard species liftover resumed/completed"

⏭️  Already done: Gorilla Adipocytes — skipping

⏭️  Already done: Chimpanzee Adipocytes — skipping

⏭️  Already done: Bonobo Adipocytes — skipping

⏭️  Already done: Macaque Adipocytes — skipping

⏭️  Already done: Gorilla BEST4+_cells — skipping

⏭️  Already done: Chimpanzee BEST4+_cells — skipping

⏭️  Already done: Bonobo BEST4+_cells — skipping

⏭️  Already done: Macaque BEST4+_cells — skipping

⏭️  Already done: Gorilla Colonocytes — skipping

⏭️  Already done: Chimpanzee Colonocytes — skipping

⏭️  Already done: Bonobo Colonocytes — skipping

⏭️  Already done: Macaque Colonocytes — skipping

⏭️  Already done: Gorilla Crypt_Fibroblasts_WNT2B+ — skipping

⏭️  Already done: Chimpanzee Crypt_Fibroblasts_WNT2B+ — skipping

⏭️  Already done: Bonobo Crypt_Fibroblasts_WNT2B+ — skipping

⏭️  Already done: Macaque Crypt_Fibroblasts_WNT2B+ — skipping

⏭️  Already done: Gorilla ECs — skipping

⏭️  Already done: Chimpanzee ECs — skipping

⏭️  Already done: Bonobo ECs — skipping

⏭️  Already 

In [ ]:
# Liftover for standard species (single-step)
STANDARD_SPECIES=("Gorilla" "Chimpanzee" "Bonobo" "Macaque")

for cell_type in "${CELL_TYPES[@]}"; do
    cell_type_lower=$(echo "$cell_type" | tr '[:upper:]' '[:lower:]')
    
    for species in "${STANDARD_SPECIES[@]}"; do
        species_lower=$(echo "$species" | tr '[:upper:]' '[:lower:]')
        
        input_file="fragment_files/${cell_type_lower}/${species_lower}_${cell_type_lower}.fragments.tsv.gz"
        chain_file="${CHAIN_FILES[$species]}"
        output_file="lifted_fragments/${cell_type_lower}/${species_lower}_${cell_type_lower}.hg38.fragments_par.tsv.gz"
        log_file="lifted_fragments/${cell_type_lower}/${species_lower}_${cell_type_lower}.log"
        
        if [[ -f "$input_file" && -f "$chain_file" ]]; then
            echo "🚀 Running liftover for ${species} ${cell_type}..."
            bash liftover_fragments_par.sh \
                --i "$input_file" \
                --c "$chain_file" \
                --o "$output_file" \
                --ncpu 30 \
                &> "$log_file"
            echo "✅ Done with ${species} ${cell_type}"
        else
            echo "⚠️  Skipping ${species} ${cell_type} (missing input or chain file)"
        fi
        echo ""
    done
done

echo "✅ Standard species liftover completed"

🚀 Running liftover for Gorilla Adipocytes...
✅ Done with Gorilla Adipocytes

🚀 Running liftover for Chimpanzee Adipocytes...
✅ Done with Chimpanzee Adipocytes

🚀 Running liftover for Bonobo Adipocytes...
✅ Done with Bonobo Adipocytes

🚀 Running liftover for Macaque Adipocytes...
✅ Done with Macaque Adipocytes

🚀 Running liftover for Gorilla BEST4+_cells...
✅ Done with Gorilla BEST4+_cells

🚀 Running liftover for Chimpanzee BEST4+_cells...
✅ Done with Chimpanzee BEST4+_cells

🚀 Running liftover for Bonobo BEST4+_cells...
✅ Done with Bonobo BEST4+_cells

🚀 Running liftover for Macaque BEST4+_cells...
✅ Done with Macaque BEST4+_cells

🚀 Running liftover for Gorilla Colonocytes...
✅ Done with Gorilla Colonocytes

🚀 Running liftover for Chimpanzee Colonocytes...


### Marmoset Two-Step Liftover

Marmoset requires two liftover steps:
1. calJac1 → calJac4
2. calJac4 → hg38

In [6]:
# Marmoset liftover - Step 1: calJac1 to calJac4
for cell_type in "${CELL_TYPES[@]}"; do
    cell_type_lower=$(echo "$cell_type" | tr '[:upper:]' '[:lower:]')
    
    input_file="fragment_files/${cell_type_lower}/marmoset_${cell_type_lower}.fragments.tsv.gz"
    chain_file="${CHAIN_FILES[Marmoset_step1]}"
    output_file="lifted_fragments/${cell_type_lower}/marmoset_${cell_type_lower}.calJac4.fragments_par.tsv.gz"
    log_file="lifted_fragments/${cell_type_lower}/marmoset_${cell_type_lower}_step1.log"
    
    # --- CHECK 1: Skip if output exists and is valid ---
    if [[ -f "$output_file" ]]; then
        # Check if the gzip file is intact and not corrupt
        if gzip -t "$output_file" 2>/dev/null; then
            echo "⏭️  Skipping Marmoset ${cell_type} Step 1 (Output exists and is complete)"
            continue
        else
            echo "⚠️  Output file for ${cell_type} exists but is corrupt/incomplete. Re-running..."
        fi
    fi
    
    # --- CHECK 2: Verify inputs exist ---
    if [[ -f "$input_file" && -f "$chain_file" ]]; then
        echo "🚀 Running Marmoset ${cell_type} liftover - Step 1 (calJac1 → calJac4)..."
        bash liftover_fragments_parchr.sh \
            --i "$input_file" \
            --c "$chain_file" \
            --o "$output_file" \
            --ncpu 30 \
            &> "$log_file"
        echo "✅ Done with Marmoset ${cell_type} Step 1"
    else
        echo "⚠️  Skipping Marmoset ${cell_type} Step 1 (missing input or chain file)"
    fi
    echo ""
done

⏭️  Skipping Marmoset Adipocytes Step 1 (Output exists and is complete)
⏭️  Skipping Marmoset BEST4+_cells Step 1 (Output exists and is complete)
⏭️  Skipping Marmoset Colonocytes Step 1 (Output exists and is complete)
⏭️  Skipping Marmoset Crypt_Fibroblasts_WNT2B+ Step 1 (Output exists and is complete)
⏭️  Skipping Marmoset ECs Step 1 (Output exists and is complete)
⏭️  Skipping Marmoset EECs Step 1 (Output exists and is complete)
⏭️  Skipping Marmoset Enteric_glia Step 1 (Output exists and is complete)
⏭️  Skipping Marmoset Enteric_neurons Step 1 (Output exists and is complete)
⏭️  Skipping Marmoset Enterocytes Step 1 (Output exists and is complete)
⚠️  Skipping Marmoset Eosinophils Step 1 (missing input or chain file)

⏭️  Skipping Marmoset Goblet-like_cells Step 1 (Output exists and is complete)
⏭️  Skipping Marmoset Goblet_cells Step 1 (Output exists and is complete)
⏭️  Skipping Marmoset ICCs Step 1 (Output exists and is complete)
⚠️  Skipping Marmoset ILCs Step 1 (missing input 

In [ ]:
# Marmoset liftover - Step 2: calJac4 to hg38
for cell_type in "${CELL_TYPES[@]}"; do
    cell_type_lower=$(echo "$cell_type" | tr '[:upper:]' '[:lower:]')
    
    input_file="lifted_fragments/${cell_type_lower}/marmoset_${cell_type_lower}.calJac4.fragments_par.tsv.gz"
    chain_file="${CHAIN_FILES[Marmoset_step2]}"
    output_file="lifted_fragments/${cell_type_lower}/marmoset_${cell_type_lower}.hg38.fragments_par.tsv.gz"
    log_file="lifted_fragments/${cell_type_lower}/marmoset_${cell_type_lower}_step2.log"
    
    # --- CHECK 1: PRE-RUN (Skip if already finished and valid) ---
    # gzip -t tests the integrity of the compressed file
    if [[ -f "$output_file" ]] && gzip -t "$output_file" &>/dev/null; then
        echo "⏭️  Skipping Marmoset ${cell_type} Step 2 (Output already exists and is valid)"
        continue
    fi

    if [[ -f "$input_file" && -f "$chain_file" ]]; then
        echo "🚀 Running Marmoset ${cell_type} liftover - Step 2 (calJac4 → hg38)..."
        
        bash liftover_fragments_parchr.sh \
            --i "$input_file" \
            --c "$chain_file" \
            --o "$output_file" \
            --ncpu 40 \
            &> "$log_file"

        # --- CHECK 2: POST-RUN (Verify success) ---
        if [[ -f "$output_file" ]] && gzip -t "$output_file" &>/dev/null; then
            echo "✅ Done with Marmoset ${cell_type} Step 2 (Output verified)"
        else
            echo "❌ FAILED Marmoset ${cell_type} Step 2: Output file is missing or corrupt!"
            # Optional: remove the corrupt file so it doesn't trick the check next time
            # rm -f "$output_file" 
        fi
    else
        echo "⚠️  Skipping Marmoset ${cell_type} Step 2 (missing input or chain file)"
    fi
    echo ""
done

echo "✅ Marmoset liftover completed"

⏭️  Skipping Marmoset Adipocytes Step 2 (Output already exists and is valid)
⏭️  Skipping Marmoset BEST4+_cells Step 2 (Output already exists and is valid)
⏭️  Skipping Marmoset Colonocytes Step 2 (Output already exists and is valid)
⏭️  Skipping Marmoset Crypt_Fibroblasts_WNT2B+ Step 2 (Output already exists and is valid)
⏭️  Skipping Marmoset ECs Step 2 (Output already exists and is valid)
⏭️  Skipping Marmoset EECs Step 2 (Output already exists and is valid)
⏭️  Skipping Marmoset Enteric_glia Step 2 (Output already exists and is valid)
⏭️  Skipping Marmoset Enteric_neurons Step 2 (Output already exists and is valid)
⏭️  Skipping Marmoset Enterocytes Step 2 (Output already exists and is valid)
⚠️  Skipping Marmoset Eosinophils Step 2 (missing input or chain file)

⏭️  Skipping Marmoset Goblet-like_cells Step 2 (Output already exists and is valid)
🚀 Running Marmoset Goblet_cells liftover - Step 2 (calJac4 → hg38)...


In [ ]:
# Marmoset liftover - Step 1: calJac1 to calJac4
for cell_type in "${CELL_TYPES[@]}"; do
    cell_type_lower=$(echo "$cell_type" | tr '[:upper:]' '[:lower:]')
    
    input_file="fragment_files/${cell_type_lower}/marmoset_${cell_type_lower}.fragments.tsv.gz"
    chain_file="${CHAIN_FILES[Marmoset_step1]}"
    output_file="lifted_fragments/${cell_type_lower}/marmoset_${cell_type_lower}.calJac4.fragments_par.tsv.gz"
    log_file="lifted_fragments/${cell_type_lower}/marmoset_${cell_type_lower}_step1.log"
    
    if [[ -f "$input_file" && -f "$chain_file" ]]; then
        echo "🚀 Running Marmoset ${cell_type} liftover - Step 1 (calJac1 → calJac4)..."
        bash liftover_fragments_parchr.sh \
            --i "$input_file" \
            --c "$chain_file" \
            --o "$output_file" \
            --ncpu 30 \
            &> "$log_file"
        echo "✅ Done with Marmoset ${cell_type} Step 1"
    else
        echo "⚠️  Skipping Marmoset ${cell_type} Step 1 (missing input or chain file)"
    fi
    echo ""
done

🚀 Running Marmoset Adipocytes liftover - Step 1 (calJac1 → calJac4)...
✅ Done with Marmoset Adipocytes Step 1

🚀 Running Marmoset BEST4+_cells liftover - Step 1 (calJac1 → calJac4)...
✅ Done with Marmoset BEST4+_cells Step 1

🚀 Running Marmoset Colonocytes liftover - Step 1 (calJac1 → calJac4)...
✅ Done with Marmoset Colonocytes Step 1

🚀 Running Marmoset Crypt_Fibroblasts_WNT2B+ liftover - Step 1 (calJac1 → calJac4)...


In [ ]:
# Marmoset liftover - Step 2: calJac4 to hg38
for cell_type in "${CELL_TYPES[@]}"; do
    cell_type_lower=$(echo "$cell_type" | tr '[:upper:]' '[:lower:]')
    
    input_file="lifted_fragments/${cell_type_lower}/marmoset_${cell_type_lower}.calJac4.fragments_par.tsv.gz"
    chain_file="${CHAIN_FILES[Marmoset_step2]}"
    output_file="lifted_fragments/${cell_type_lower}/marmoset_${cell_type_lower}.hg38.fragments_par.tsv.gz"
    log_file="lifted_fragments/${cell_type_lower}/marmoset_${cell_type_lower}_step2.log"
    
    if [[ -f "$input_file" && -f "$chain_file" ]]; then
        echo "🚀 Running Marmoset ${cell_type} liftover - Step 2 (calJac4 → hg38)..."
        bash liftover_fragments_parchr.sh \
            --i "$input_file" \
            --c "$chain_file" \
            --o "$output_file" \
            --ncpu 30 \
            &> "$log_file"
        echo "✅ Done with Marmoset ${cell_type} Step 2"
    else
        echo "⚠️  Skipping Marmoset ${cell_type} Step 2 (missing input or chain file)"
    fi
    echo ""
done

echo "✅ Marmoset liftover completed"

In [ ]:
# Verify lifted fragment files
for cell_type in "${CELL_TYPES[@]}"; do
    cell_type_lower=$(echo "$cell_type" | tr '[:upper:]' '[:lower:]')
    echo "\n📂 Lifted fragments for ${cell_type}:"
    ls -lh "lifted_fragments/${cell_type_lower}/" | grep -E '\.hg38\.fragments.*\.tsv\.gz$'
done

---
## Step 3: Convert Fragments to BigWig Coverage Files

Convert lifted fragment files to coverage bigWig format for visualization.

**Note:** Requires `scatac_fragment_tools` to be installed.

In [ ]:
# Activate the scatac_fragment_tools environment
mamba activate scatac_fragment_tools

In [ ]:
# Create output directories for bigWig files (organized by cell type)
for cell_type in "${CELL_TYPES[@]}"; do
    cell_type_lower=$(echo "$cell_type" | tr '[:upper:]' '[:lower:]')
    mkdir -p "bigwigs/${cell_type_lower}"
    echo "📁 Created directory: bigwigs/${cell_type_lower}"
done

echo "✅ BigWig output directories created"

In [ ]:
# Convert fragments to bigWig for all species and cell types
# Pre-check global requirement
[[ ! -f "$CHROM_SIZES" ]] && echo "⚠️ Missing CHROM_SIZES file ($CHROM_SIZES)" && exit 1

for cell in "${CELL_TYPES[@]}"; do
    c_low="${cell,,}" 
    
    for sp in "${SPECIES[@]}"; do
        s_low="${sp,,}"
        
        # Define paths (Handle Human filename exception dynamically)
        suffix=$([[ "$sp" == "Human" ]] && echo "fragments" || echo "fragments_par")
        input="lifted_fragments/${c_low}/${s_low}_${c_low}.hg38.${suffix}.tsv.gz"
        output="bigwigs/${c_low}/${s_low}_${c_low}.hg38.cov.bw"

        # Check input existence
        [[ ! -f "$input" ]] && echo "⚠️ Missing fragments for $sp $cell" && continue

        # Check if output already exists AND is valid
        # Logic: If file exists (-s) AND (bigWigInfo is missing OR bigWigInfo says file is good)
        if [[ -s "$output" ]] && { ! command -v bigWigInfo &>/dev/null || bigWigInfo "$output" &>/dev/null; }; then
            echo "⏩ Valid bigWig exists for $sp $cell — skipping"
            continue
        fi

        # Run Conversion
        mkdir -p "$(dirname "$output")"
        echo "🎯 Converting $sp $cell..."
        
        scatac_fragment_tools bigwig -i "$input" -c "$CHROM_SIZES" -o "$output" -n

        # Final Verification
        if [[ -s "$output" ]] && { ! command -v bigWigInfo &>/dev/null || bigWigInfo "$output" &>/dev/null; }; then
            echo "✅ Done: $output"
        else
            echo "❌ Failed/Corrupt: $output"
        fi
    done
done

echo "✅ All bigWig conversions completed"

In [ ]:
# Convert fragments to bigWig for all species and cell types
for cell_type in "${CELL_TYPES[@]}"; do
    cell_type_lower=$(echo "$cell_type" | tr '[:upper:]' '[:lower:]')
    
    for species in "${SPECIES[@]}"; do
        species_lower=$(echo "$species" | tr '[:upper:]' '[:lower:]')
        
        # Determine input file name based on species
        if [[ "$species" == "Human" ]]; then
            # Human uses direct link (no _par suffix)
            input_frag="lifted_fragments/${cell_type_lower}/${species_lower}_${cell_type_lower}.hg38.fragments.tsv.gz"
        else
            # Other species use parallel liftover output (_par suffix)
            input_frag="lifted_fragments/${cell_type_lower}/${species_lower}_${cell_type_lower}.hg38.fragments_par.tsv.gz"
        fi
        
        output_bw="bigwigs/${cell_type_lower}/${species_lower}_${cell_type_lower}.hg38.cov.bw"
        
        if [[ -f "$input_frag" && -f "$CHROM_SIZES" ]]; then
            echo "🎯 Converting ${species} ${cell_type} to bigWig..."
            scatac_fragment_tools bigwig \
                -i "$input_frag" \
                -c "$CHROM_SIZES" \
                -o "$output_bw" \
                -n
            echo "✅ Done: $output_bw"
        else
            echo "⚠️  Missing input for ${species} ${cell_type} — skipping"
        fi
        echo ""
    done
done

echo "✅ All bigWig conversions completed"

In [7]:
# Convert fragments to bigWig for all species and cell types
for cell_type in "${CELL_TYPES[@]}"; do
    cell_type_lower=$(echo "$cell_type" | tr '[:upper:]' '[:lower:]')
    
    for species in "${SPECIES[@]}"; do
        species_lower=$(echo "$species" | tr '[:upper:]' '[:lower:]')
        
        # Determine input file name based on species
        if [[ "$species" == "Human" ]]; then
            # Human uses direct link (no _par suffix)
            input_frag="lifted_fragments/${cell_type_lower}/${species_lower}_${cell_type_lower}.hg38.fragments.tsv.gz"
        else
            # Other species use parallel liftover output (_par suffix)
            input_frag="lifted_fragments/${cell_type_lower}/${species_lower}_${cell_type_lower}.hg38.fragments_par.tsv.gz"
        fi
        
        output_bw="bigwigs/${cell_type_lower}/${species_lower}_${cell_type_lower}.hg38.cov.bw"

        # Check inputs
        if [[ ! -f "$input_frag" ]]; then
            echo "⚠️  Missing fragments for ${species} ${cell_type} — skipping"
            echo ""
            continue
        fi
        if [[ ! -f "$CHROM_SIZES" ]]; then
            echo "⚠️  Missing CHROM_SIZES file ($CHROM_SIZES) — skipping all"
            exit 1
        fi

        # If bigWig already exists, check if it's valid
        if [[ -s "$output_bw" ]]; then
            if command -v bigWigInfo &>/dev/null; then
                if bigWigInfo "$output_bw" >/dev/null 2>&1; then
                    echo "⏩ Existing valid bigWig for ${species} ${cell_type} — skipping"
                    echo ""
                    continue
                else
                    echo "♻️ Existing bigWig for ${species} ${cell_type} seems corrupt — regenerating"
                fi
            else
                echo "ℹ️ bigWigInfo not found. Existing bigWig for ${species} ${cell_type} detected — assuming OK and skipping."
                echo ""
                continue
            fi
        fi

        # Ensure output directory exists
        mkdir -p "$(dirname "$output_bw")"

        echo "🎯 Converting ${species} ${cell_type} to bigWig..."
        scatac_fragment_tools bigwig \
            -i "$input_frag" \
            -c "$CHROM_SIZES" \
            -o "$output_bw" \
            -n

        if [[ -s "$output_bw" ]]; then
            # Optional: re-check validity after generation if bigWigInfo is available
            if command -v bigWigInfo &>/dev/null; then
                if bigWigInfo "$output_bw" >/dev/null 2>&1; then
                    echo "✅ Done & validated: $output_bw"
                else
                    echo "❌ Generated bigWig appears invalid for ${species} ${cell_type} — check logs"
                fi
            else
                echo "✅ Done: $output_bw"
            fi
        else
            echo "❌ Failed to create bigWig for ${species} ${cell_type} — empty or missing output"
        fi
        echo ""
    done
done

echo "✅ All bigWig conversions completed"

⏩ Existing valid bigWig for Human Adipocytes — skipping

⏩ Existing valid bigWig for Gorilla Adipocytes — skipping

⏩ Existing valid bigWig for Chimpanzee Adipocytes — skipping

⏩ Existing valid bigWig for Bonobo Adipocytes — skipping

⏩ Existing valid bigWig for Macaque Adipocytes — skipping

⏩ Existing valid bigWig for Marmoset Adipocytes — skipping

⏩ Existing valid bigWig for Human BEST4+_cells — skipping

⏩ Existing valid bigWig for Gorilla BEST4+_cells — skipping

⏩ Existing valid bigWig for Chimpanzee BEST4+_cells — skipping

⏩ Existing valid bigWig for Bonobo BEST4+_cells — skipping

⏩ Existing valid bigWig for Macaque BEST4+_cells — skipping

⏩ Existing valid bigWig for Marmoset BEST4+_cells — skipping

⏩ Existing valid bigWig for Human Colonocytes — skipping

⏩ Existing valid bigWig for Gorilla Colonocytes — skipping

⏩ Existing valid bigWig for Chimpanzee Colonocytes — skipping

⏩ Existing valid bigWig for Bonobo Colonocytes — skipping

⏩ Existing valid bigWig for Macaque Co

In [8]:
# Verify bigWig files
for cell_type in "${CELL_TYPES[@]}"; do
    cell_type_lower=$(echo "$cell_type" | tr '[:upper:]' '[:lower:]')
    echo "\n📂 BigWig files for ${cell_type}:"
    ls -lh "bigwigs/${cell_type_lower}/"
done

\n📂 BigWig files for Adipocytes:
total 108M
-rw-r----- 1 jjanssens jjanssens-group  46M Nov 12 06:47 bonobo_adipocytes.hg38.cov.bw
-rw-r----- 1 jjanssens jjanssens-group  12M Nov 12 06:47 chimpanzee_adipocytes.hg38.cov.bw
-rw-r----- 1 jjanssens jjanssens-group  10M Nov 12 06:47 gorilla_adipocytes.hg38.cov.bw
-rw-r----- 1 jjanssens jjanssens-group  15M Nov 12 06:47 human_adipocytes.hg38.cov.bw
-rw-r----- 1 jjanssens jjanssens-group 8.2M Nov 12 06:48 macaque_adipocytes.hg38.cov.bw
-rw-r----- 1 jjanssens jjanssens-group  18M Nov 12 06:48 marmoset_adipocytes.hg38.cov.bw
\n📂 BigWig files for BEST4+_cells:
total 272M
-rw-r----- 1 jjanssens jjanssens-group  16M Nov 12 06:49 bonobo_best4+_cells.hg38.cov.bw
-rw-r----- 1 jjanssens jjanssens-group  12M Nov 12 06:49 chimpanzee_best4+_cells.hg38.cov.bw
-rw-r----- 1 jjanssens jjanssens-group  15M Nov 12 06:49 gorilla_best4+_cells.hg38.cov.bw
-rw-r----- 1 jjanssens jjanssens-group 173M Nov 12 06:48 human_best4+_cells.hg38.cov.bw
-rw-r----- 1 jjanssen

---
## Summary

### Output Structure

```
fragment_files/
└── enterocytes/
    ├── human_enterocytes.fragments.tsv.gz
    ├── gorilla_enterocytes.fragments.tsv.gz
    ├── chimpanzee_enterocytes.fragments.tsv.gz
    ├── bonobo_enterocytes.fragments.tsv.gz
    ├── macaque_enterocytes.fragments.tsv.gz
    └── marmoset_enterocytes.fragments.tsv.gz

lifted_fragments/
└── enterocytes/
    ├── human_enterocytes.hg38.fragments.tsv.gz
    ├── gorilla_enterocytes.hg38.fragments_par.tsv.gz
    ├── chimpanzee_enterocytes.hg38.fragments_par.tsv.gz
    ├── bonobo_enterocytes.hg38.fragments_par.tsv.gz
    ├── macaque_enterocytes.hg38.fragments_par.tsv.gz
    └── marmoset_enterocytes.hg38.fragments_par.tsv.gz

bigwigs/
└── enterocytes/
    ├── human_enterocytes.hg38.cov.bw
    ├── gorilla_enterocytes.hg38.cov.bw
    ├── chimpanzee_enterocytes.hg38.cov.bw
    ├── bonobo_enterocytes.hg38.cov.bw
    ├── macaque_enterocytes.hg38.cov.bw
    └── marmoset_enterocytes.hg38.cov.bw
```

### Processing Notes

- **Human**: No liftover needed (already hg38), direct symlink created
- **Standard species** (Gorilla, Chimpanzee, Bonobo, Macaque): Single-step liftover with 30 CPUs
- **Marmoset**: Two-step liftover (calJac1 → calJac4 → hg38) with 30 CPUs
- All outputs organized by cell type for easy access and management
- BigWig files use normalized coverage (`-n` flag) for cross-sample comparison